<a href="https://colab.research.google.com/github/greyhound101/Orbuculum/blob/main/resize_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
import zipfile
import pandas as pd
import glob
import os
import cv2
from tqdm import tqdm
import random

In [3]:
import zipfile
with zipfile.ZipFile('/content/gdrive/MyDrive/GTSRB_Final_Training_Images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [4]:
cols=['Filename','Width','Height','Roi.X1','Roi.Y1','Roi.X2','Roi.Y2','ClassId']
for name in glob.glob('/content/GTSRB/Final_Training/Images/*/GT-*.csv'):
  df=pd.read_csv(name)
  dk={'Filename':[],'Width':[],'Height':[],'Roi.X1':[],'Roi.Y1':[],'Roi.X2':[],'Roi.Y2':[],'ClassId':[]}
  for i in df.iterrows():
    for j in range(8):
      dk[cols[j]].append(i[1][0].split(';')[j])
  name=name.replace( "input", "working" )
  pd.DataFrame(dk).to_csv(name.split('/')[-1],index=False)

In [5]:
for en,path in enumerate(glob.glob('GT-*.csv')):
  df=pd.read_csv(path)
  if en==0:
    fnl=df
  else:
    fnl=pd.concat([df,fnl],0)
fnl['x_diff']=fnl['Roi.X2']-fnl['Roi.X1']
fnl['y_diff']=fnl['Roi.Y2']-fnl['Roi.Y1']
fnl['x_diff'].median(),fnl['y_diff'].median()

(33.0, 32.0)

In [8]:
for clas in tqdm(glob.glob('*.csv')):
  clas=clas.split('-')[-1].split('.')[0]
  df=pd.read_csv('GT-'+clas+'.csv')
  for i in df.iterrows():
    val=i[1].values
    name=val[0]
    x1=val[3]
    x2=val[5]
    y1=val[4]
    y2=val[6]
    total_name='/content/GTSRB/Final_Training/Images/'+clas+'/'+name
    img=cv2.imread(total_name)
    try:
      os.mkdir('/content/gdrive/MyDrive/orc/class_'+clas)
    except:
        pass
    cv2.imwrite('/content/gdrive/MyDrive/orc/class_'+clas+'/'+name,cv2.resize(img[x1:x2,y1:y2,:],(30,30), interpolation = cv2.INTER_AREA))


100%|██████████| 43/43 [04:23<00:00,  6.14s/it]


In [14]:
00000

[]

In [15]:
dk={}
for en,clas in enumerate(glob.glob('/content/gdrive/MyDrive/orc/class_*')):
  clas=clas.split('_')[-1]
  dk[clas]={'train':[],'test':[],'validation':[]}
  
  options=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/*_00000.ppm')
  options=[a.split('/')[-1].split('_')[0] for a in options]
  
  ln=len(options)
  all=list(range(ln))
  test_val=round(0.2*ln)
  
  test=random.sample(all, test_val)
  for i in test:
    all.remove(i)
  test=[options[i] for i in test]
  for en,i in enumerate(test):
    if en==0:  
        dk[clas]['test']=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/'+i+'_*.ppm')
    else:
        dk[clas]['test']+=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/'+i+'_*.ppm')
        
    
  validation=random.sample(all, test_val)
  for i in validation:
    all.remove(i)
  validation=[options[i] for i in validation]
  for en,i in enumerate(validation):
    if en==0:  
        dk[clas]['validation']=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/'+i+'_*.ppm')
    else:
        dk[clas]['validation']+=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/'+i+'_*.ppm')
    
  
  train=[options[i] for i in all]
  for en,i in enumerate(train):
    if en==0:  
        dk[clas]['train']=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/'+i+'_*.ppm')
    else:
        dk[clas]['train']+=glob.glob( '/content/gdrive/MyDrive/orc/class_'+clas+'/'+i+'_*.ppm')

In [16]:
df=pd.DataFrame(dk).T

In [21]:
df.head()

,train,test,validation
00042,[/content/gdrive/MyDrive/orc/class_00042/00001...,[/content/gdrive/MyDrive/orc/class_00042/00005...,[/content/gdrive/MyDrive/orc/class_00042/00000...
00022,[/content/gdrive/MyDrive/orc/class_00022/00001...,[/content/gdrive/MyDrive/orc/class_00022/00003...,[/content/gdrive/MyDrive/orc/class_00022/00012...
00036,[/content/gdrive/MyDrive/orc/class_00036/00000...,[/content/gdrive/MyDrive/orc/class_00036/00004...,[/content/gdrive/MyDrive/orc/class_00036/00010...
00040,[/content/gdrive/MyDrive/orc/class_00040/00000...,[/content/gdrive/MyDrive/orc/class_00040/00010...,[/content/gdrive/MyDrive/orc/class_00040/00002...
00002,[/content/gdrive/MyDrive/orc/class_00002/00000...,[/content/gdrive/MyDrive/orc/class_00002/00053...,[/content/gdrive/MyDrive/orc/class_00002/00017...


In [18]:
df.to_csv('/content/gdrive/My Drive/orc/split.csv')